In [3]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-06-25 18:18:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2023-06-25 18:18:48 (8.51 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [4]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
len(text)

1115394

In [6]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)

In [8]:
print("".join(vocab), vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [9]:
# Create a Mapping -- simple char to int mapping
# Map str to int -- stoi
stoi, itos = {}, {}
for i, ch in enumerate(vocab):
    stoi[ch] = i
for i, ch in enumerate(vocab):
    itos[i] = ch

In [10]:
print(stoi, itos)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64} {0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [11]:
# Encoding and Decoding
# Encoder i/p: string, o/p: int
def encode(s):
    _stoi = []
    for ch in s:
        _stoi.append(stoi[ch])
    return _stoi

# Decoder i/p: int, o/p: str
def decode(i:int)->str:
    _itos = []
    for _int in i:
        _itos.append(itos[_int])
    return "".join(_itos)

In [12]:
print(encode("You are being watched"))
print(decode(encode("You are being watched")))

[37, 53, 59, 1, 39, 56, 43, 1, 40, 43, 47, 52, 45, 1, 61, 39, 58, 41, 46, 43, 42]
You are being watched


In [13]:
# Encode the entire dataset
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
data = torch.tensor(encode(text), dtype=torch.long, device=device)

In [14]:
data.shape

torch.Size([1115394])

In [15]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59], device='cuda:0')

In [16]:
# Train and val set
seed = 108
split = 0.9
n = int(len(data) * split) 
# TODO: Add shuffle
# data = torch.shuffle(data, seed=seed)
train_data = data[:n]
test_data = data[n:]

In [17]:
# Max len of chunks in train data to look at
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58], device='cuda:0')

In [27]:
X = train_data[:block_size+1]
y = train_data[1:block_size+1]
print(f"X: {X}, y: {y}")
for i in range(block_size):
    context = X[:i+1]
    target = y[i]
    print(f"Input context: {context} with target:{target} ")

X: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58], device='cuda:0'), y: tensor([47, 56, 57, 58,  1, 15, 47, 58], device='cuda:0')
Input context: tensor([18], device='cuda:0') with target:47 
Input context: tensor([18, 47], device='cuda:0') with target:56 
Input context: tensor([18, 47, 56], device='cuda:0') with target:57 
Input context: tensor([18, 47, 56, 57], device='cuda:0') with target:58 
Input context: tensor([18, 47, 56, 57, 58], device='cuda:0') with target:1 
Input context: tensor([18, 47, 56, 57, 58,  1], device='cuda:0') with target:15 
Input context: tensor([18, 47, 56, 57, 58,  1, 15], device='cuda:0') with target:47 
Input context: tensor([18, 47, 56, 57, 58,  1, 15, 47], device='cuda:0') with target:58 


In [28]:
train_data.shape

torch.Size([1003854])

In [48]:
# Create a dataloader
batch_size = 4
block_size = 8
# Random seed
torch.manual_seed(1337)
X = []
y = []
def get_batch(split:str):
    data = train_data if split=="train" else test_data
    # Random offsets in data
    idx = torch.randint(low=0,high=(len(data)-block_size), size=(batch_size,))
    X = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+1+block_size] for i in idx])
    return X, y

In [60]:
xb, yb = get_batch("train")


In [61]:
print(xb.shape)
# 4x8 -> 4 batch, 8 block size

torch.Size([4, 8])


In [62]:
print(xb)

tensor([[ 6,  0, 21, 44,  1, 61, 43,  1],
        [58, 52, 43, 57, 57,  2,  1, 57],
        [ 1, 59, 52, 39, 41, 46, 47, 52],
        [43, 42, 50, 39, 56,  6,  1, 50]], device='cuda:0')


In [68]:
# Bigram Model 
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size, device=device)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) # Batch = 4, Time = 8(context), Channel=65(Vocab Size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # cross_entropy wants B, C, T instead B, T, C
            logits = logits.view(B*T, C) # Stretch in 2D
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # Negative log likelihood

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8014, device='cuda:0', grad_fn=<NllLossBackward0>)

JrTxbDkRZkNwc.wj,ZTxO-On-y$WK
baqPe?kMBFeA$G:XZSGgO-3cjMGd?gLhaGhX'YVX3tpgfNuwq&$WWv.tbaF :X3!FHaGeN
